# Reading FORTRAN generated binary output
## NCODA Pre-QC

Includes references to plotting using Matplotlib and related tools.

This program reads in a single input file after definine a binary "struct" with which to read in the data.

In [1]:
#Establish setup for graphics, appropriate for jupyter lab
%matplotlib inline
#use nbagg for interactivity
#%matplotlib nbagg

In [2]:
BUCKET_NAME     = "ai-training-2024-08-09-bucket"
PROJECT_ID      = "ai-training-2024-08-09"
LOCATION        = "us-central1"
secret_name     = "ai-training-key-secret"
secret_version  = "latest"
project_id      = "usfs-tf-admin"
resource_name   = f"projects/{project_id}/secrets/{secret_name}/versions/{secret_version}"

In [3]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#- Google Colab Check
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
RunningInCOLAB = False
RunningInCOLAB = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    print("You are running this notebook in Google Colab.")
else:
    print("You are running this notebook with Jupyter iPython runtime.")
    print("Assumption is you have the required libraries to execute this notebook.")

You are running this notebook in Google Colab.


In [4]:
import sys
import subprocess
import importlib.util

In [5]:
libraries=["numpy", "pandas", "scipy", "sklearn", "matplotlib", "xarray", "seaborn", "cartopy", "owslib"]
import importlib.util

for library in libraries:
    if library == "Pillow":
      spec = importlib.util.find_spec("PIL")
    else:
      spec = importlib.util.find_spec(library)
    if spec is None:
      print("Installing library " + library)
      subprocess.run(["pip", "install" , library, "--quiet"])
    else:
      print("Library " + library + " already installed.")

Library numpy already installed.
Library pandas already installed.
Library scipy already installed.
Library sklearn already installed.
Library matplotlib already installed.
Library xarray already installed.
Library seaborn already installed.
Installing library cartopy
Installing library owslib


# Includes and Libraries

In [16]:
############################################
# INCLUDES
############################################
#libraries specific to this example
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.linear_model import LinearRegression

#plotting
import matplotlib as matplt
import matplotlib.pyplot as plt
#geographic libraries
from cartopy import crs as ccrs
from matplotlib.offsetbox import AnchoredText
from shapely.geometry import Polygon
import owslib
import geopandas as gd

#byte manipulation and endianess
import sys

# for NetCDF output
from scipy.io import netcdf

#a set of libraries that perhaps should always be in Python source
import os
from datetime import datetime
import sys
import gc
import getopt
import inspect
import math
import warnings

from pydoc import help                          # can type in the python console `help(name of function)` to get the documentation

#a darn useful library for creating paths and one I recommend you load to your environment
from pathlib import Path



warnings.filterwarnings('ignore')               # don't print out warnings


# Pull in Library Functions

In [7]:
#!rm -rf ./folderOnColab && echo "Ok, removed." || { echo "No folder to remove."; exit 1; }
#!mkdir -p ./folderOnColab && echo "Folder created." || { echo "Failed to create folder, it might already exist.";  }
#!gsutil -m cp -r gs://usfs-gcp-rand-test-data-usc1/public_source/jbooks/ANewHope.txt ./folderOnColab

target_folder="./folderOnColab"
target_repo="https://raw.githubusercontent.com//christophergarthwood/jbooks/main"
target_files=["support_debug.ipynb", "support_functions.ipynb"]
print(f"Creating a folder ({target_folder}) to store project data.")
subprocess.run(["mkdir", "-p" , target_folder])
if os.path.isdir(target_folder):
  print("Performing wget on:")
  for idx, filename in enumerate(target_files):
    print(f"...{filename} to target folder: {target_folder}")
    try:
      subprocess.run(["wget", f"--directory-prefix={target_folder}", f"{target_repo}/{filename}"])
    except Exception as e:
      print("")
      print(f"ERROR: There was a problem performing wget on the target file ({filename}), see Exception: {str(e)}")
      print("...talk to the instructor.")
    if os.path.isfile(target_folder+os.sep+filename):
      print("...verified copy.")
      print("...importing code.")
      target_filename=f"{target_folder+os.sep+filename}"
      os.environ["target_filename"]=target_filename
      %run $target_filename
    else:
      print(f"...copy NOT verified, check the {target_folder} for the existence of {filename}")
else:
    print("ERROR: Local folder not found/created.  Check the output to ensure your folder is created.")
    print(f"...target folder: {target_folder}")
    print("...if you can't find the problem contact the instructor.")


Creating a folder (./folderOnColab) to store project data.
Performing wget on:
...support_debug.ipynb to target folder: ./folderOnColab
...verified copy.
...importing code.
...support_functions.ipynb to target folder: ./folderOnColab
...verified copy.
...importing code.


# Setup Library (Pandas/Numpy) Configuration

In [8]:
msg_info("Setting Library Configuration")
set_library_configuration()

[2024-08-27 23:08:44 UTC]    INFO: Setting Library Configuration 


INFO:msg_logs:Setting Library Configuration


# Variable declaration

In [9]:
############################################
# GLOBAL VARIABLES
############################################
DEBUG = 1
DEBUG_DATA = 0

# CODE CONSTRAINTS
VERSION_NAME    = "NCODA_PREQC"
VERSION_MAJOR   = 0
VERSION_MINOR   = 0
VERSION_RELEASE = 1

#used for values outside standard ASCII, just do it, you'll need it
ENCODING  ="utf-8"

############################################
# GLOBAL CONSTANTS
############################################


############################################
# APPLICATION VARIABLES
############################################

#variable persistence
today = datetime.today()

############################################
# GLOBAL CONFIGURATION
############################################
os.environ['PYTHONIOENCODING']=ENCODING


# Check your library versions

In [10]:
msg_info("Library Diagnostics")
lib_diagnostics()

[2024-08-27 23:08:44 UTC]    INFO: Library Diagnostics 


INFO:msg_logs:Library Diagnostics


matplotlib                              #: 3.7.1               
numpy                                   #: 1.26.4              
pandas                                  #: 2.1.4               
seaborn                                 #: 0.13.1              
xarray                                  #: 2024.6.0            


# Data

In [11]:
#!rm -rf ./folderOnColab && echo "Ok, removed." || { echo "No folder to remove."; exit 1; }
#!mkdir -p ./folderOnColab && echo "Folder created." || { echo "Failed to create folder, it might already exist.";  }
#!gsutil -m cp -r gs://usfs-gcp-rand-test-data-usc1/public_source/jbooks/ANewHope.txt ./folderOnColab

target_folder="./folderOnColab"
target_files=["merged.2020032.0201.L5.6D.a_490_lmi.nc_202001270400_a_490_lmi_2.out"]
print(f"Creating a folder ({target_folder}) to store project data.")
subprocess.run(["mkdir", "-p" , target_folder])
if os.path.isdir(target_folder):
  for idx, filename in enumerate(target_files):
    print(f"Copying {filename} to target folder: {target_folder}")
    subprocess.run(["gsutil", "-m" , "cp", "-r", f"gs://{BUCKET_NAME}/public_source/jbooks/{filename}",  target_folder], check=True)
else:
    print("ERROR: Local folder not found/created.  Check the output to ensure your folder is created.")
    print(f"...target folder: {target_folder}")
    print("...if you can't find the problem contact the instructor.")


Creating a folder (./folderOnColab) to store project data.
Copying merged.2020032.0201.L5.6D.a_490_lmi.nc_202001270400_a_490_lmi_2.out to target folder: ./folderOnColab


# GOPS Parser Slice Output

In [12]:
#################################################################################
#CONSTANTS
#################################################################################

#Notice that this is a 300 MB binary file which is NOT included in these training sets.
DATA_DIR=Path(target_folder+os.sep)

#NRL employees
#DATA_DIR=Path("/u/yamm/cwood/outgoing/VISOR")

DATA_FILE=["merged.2020032.0201.L5.6D.a_490_lmi.nc_202001270400_a_490_lmi_2.out"]

#each record in FORTRAN "bounded" by a value that specifies how big the next block to read is hence the RB or record boundary value
FORTRAN_RB_CHUNK_SIZE=4

#expressly defining data types as the original FORTRAN implementation is specific as well
FORTRAN_INT_CHUNK_SIZE=4
FORTRAN_REAL_CHUNK_SIZE=4
DATA_CHUNK=0
ENDIANESS='big'

#Data formatting (least significant digit)
LSD=4

#Geospatial filter criteria, known for this specific data file (Gulf of Mexico)
FLTR_NW_LAT=31.0519
FLTR_NW_LON=-98.4798
FLTR_SE_LAT=19.2310
FLTR_SE_LON=-80.9669
FLTR_DATA = ("lat > " + str(FLTR_SE_LAT) + " & lat < " + str(FLTR_NW_LAT) + " & lon > " + str(FLTR_NW_LON) + " & lon < " + str(FLTR_SE_LON) )

#define data structure of the NCODA-PreQC/VISOR output
preQC_datatype=np.dtype([('frb_nrec1', '>i4'),
                         ('nrec','>i4'),
                         ('frb_nrec2', '>i4'),
                         ('frb_dtype1', '>i4'),
                         ('dtype','>i4'),
                         ('frb_dtype2', '>i4'),
                         ('frb_dtg1', '>i4'),
                         ('dtg','>a12'),
                         ('frb_dtg2', '>i4')
                        ])

In [13]:
#create a function for dynamic Polygon creation
def get_polygon(df, offset):
    #find the geographic boundaries of the total dataset
    bounds_df = df.bounds
    nlat = bounds_df['maxy'].max() + offset
    slat = bounds_df['miny'].max() - offset
    elon = bounds_df['maxx'].max() + offset
    wlon = bounds_df['minx'].max() - offset
    #report those boundaries
    msg_info("------------------------ COORDINATES ------------------------")
    msg_debug("  Northern Latitude:{:8.4f}".format(nlat))
    msg_debug("  Southern Latitude:{:8.4f}".format(slat))
    msg_debug(" Northern Longitude:{:8.4f}".format(wlon))
    msg_debug(" Northern Longitude:{:8.4f}".format(elon))
    msg_debug(" Polygon( [ ({:8.4f},{:8.4f}), ({:8.4f},{:8.4f}), ({:8.4f},{:8.4f}), ({:8.4f},{:8.4f}), ({:8.4f},{:8.4f}) ])".format(wlon,nlat,wlon,slat,elon,slat,elon,nlat,wlon,nlat))
    #create a polygon for highlighting the data
    #polygon = Polygon([(0, 0), (0, 90), (180, 90), (180, 0), (0, 0)])
    polygon = Polygon([ (wlon,nlat), (wlon,slat), (elon,slat), (elon,nlat), (wlon,nlat) ])
    #poly_gdf = geopandas.GeoDataFrame([1], geometry=[polygon], crs=world.crs)
    poly_gdf = gd.GeoDataFrame([1], geometry=[polygon], crs=df.crs)

    return poly_gdf

In [14]:
#################################################################################
#Mapping routines to show where data resides (leaves much to be desired and
#doesn't make use of 008_mapping*.ipynb.
#################################################################################
def show_map_coverage(inc_dataframe):

  TARGET_DATAFRAME = inc_dataframe

  central_longitude = np.median(TARGET_DATAFRAME["lon"])
  central_latitude = np.median(TARGET_DATAFRAME["lat"])
  west = np.min(TARGET_DATAFRAME["lon"]) - 1.5
  east = np.max(TARGET_DATAFRAME["lon"]) + 1.5
  north = np.max(TARGET_DATAFRAME["lat"]) + 1.5
  south = np.min(TARGET_DATAFRAME["lat"]) - 1.5

  #let's define some basic variables
  highlight_box='red'
  highlight_color='blue'
  color_map='Spectral'
  focus_area='Gulf of Mexico'
  offset=5                    #degrees
  show_plot=1
  target_column="optics"

  #labeling
  # Add a text annotation for the license information to the bottom right corner.
  font_size=8
  SOURCE = 'Natural Earth'
  LICENSE = 'public domain'
  text = AnchoredText(r'$\mathcircled{{c}}$ {}; license: {}'
                      ''.format(SOURCE, LICENSE),
                      loc=4, prop={'size': font_size}, frameon=True)


  #GeoDataFrame needs a shapely object. We use geopandas points_from_xy() to transform Longitude and Latitude into a list of shapely.Point objects and set it as a geometry while creating the GeoDataFrame.
  gdf = gd.GeoDataFrame(TARGET_DATAFRAME, geometry=gd.points_from_xy(TARGET_DATAFRAME.Longitude, TARGET_DATAFRAME.Latitude))
  #let's make the coordinate system explicit, normally you don't have to if showing data quickly
  gdf.crs = "EPSG:3857"

  #create a polygon to "bound" the data graphically
  poly_gdf=get_polygon(gdf, offset)

  fig1=plt.figure(figsize=(8.5,11))
  fig1.suptitle('Optics from Binary File')

  ############################################
  # AX2, Upper right
  ############################################
  from cartopy.io.img_tiles import Stamen
  tiler=Stamen('terrain-background')

  ax2=fig1.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax2.set_title("Gulf of Mexico");

  increased_offset=10
  boundary=gdf
  #Western Longitude,
  ax2.set_extent([boundary.total_bounds[0]-increased_offset, boundary.total_bounds[2]+increased_offset, boundary.total_bounds[1]-increased_offset, boundary.total_bounds[3]+increased_offset], crs=ccrs.PlateCarree())
  xlim = ([boundary.total_bounds[0]-increased_offset,  boundary.total_bounds[2]+increased_offset])
  ylim = ([boundary.total_bounds[1]-increased_offset,  boundary.total_bounds[3]+increased_offset])
  ax2.set_xlim(xlim)
  ax2.set_ylim(ylim)
  #6 = terrain, it's in the example, no idea
  ax2.add_image(tiler,6)
  ax2.coastlines('10m')
  ax2.add_feature(cfeature.LAND)
  ax2.add_feature(cfeature.OCEAN)
  ax2.add_feature(cfeature.COASTLINE)
  ax2.add_feature(cfeature.BORDERS, linestyle=':')
  ax2.add_feature(cfeature.LAKES, alpha=0.5)
  ax2.add_feature(cfeature.RIVERS)

  gdf.plot(ax=ax2, color=highlight_color)
  poly_gdf=get_polygon(gdf, 1)
  poly_gdf.boundary.plot(ax=ax2, color=highlight_box)

  plt.show()



In [ ]:
#loop through the datafiles provided
for file in DATA_FILE:
    target_input_file=Path(DATA_DIR / file)
    netcdf_filename=Path(DATA_DIR / str(file + ".nc") )

    #meta-data capture from filename (merged_2019317_201911130400_a_490_lmi_2)
    file_meta_data = file.split('_')
    product_name="".join(file_meta_data[3:5:1])
    msg_debug("Processing " + str(product_name) + " for "+ str(target_input_file))

    in_file=open(target_input_file, 'rb')

    #################################################################################
    #Binary read using the pre-defined structure
    #################################################################################
    x=np.fromfile(in_file, dtype=preQC_datatype,count=1)
    DATA_CHUNK= FORTRAN_REAL_CHUNK_SIZE * x['nrec']
    msg_debug(x)
    msg_debug("Total records to read in for each array:"+str(x['nrec'][0]))
    msg_debug("Total bytes to read in for each array:"+str(DATA_CHUNK))

    number_records=str(x['nrec'][0])
    array_structure=">"+ number_records +"f4"
    array_dtype=np.dtype([('frb_ary1', '>i4'),('array',array_structure),('frb_ary2', '>i4')])

    msg_info("...processing lat data from PreQC")
    y=np.fromfile(in_file, dtype=array_dtype,count=1)
    array=y['array']
    msg_debug("......statistics:")
    msg_debug(".........min:" + str(np.nanmin(array)))
    msg_debug(".........max:" + str(np.nanmax(array)))
    msg_debug("........mean:" + str(np.nanmean(array)))
    lat=np.array(array).byteswap().newbyteorder()
    msg_debug("Lat size:"+str(lat.size))

    msg_info("...processing lon data from PreQC")
    y=np.fromfile(in_file, dtype=array_dtype,count=1)
    array=y['array']
    msg_debug("......statistics:")
    msg_debug(".........min:" + str(np.nanmin(array)))
    msg_debug(".........max:" + str(np.nanmax(array)))
    msg_debug("........mean:" + str(np.nanmean(array)))
    lon=np.array(array).byteswap().newbyteorder()
    msg_debug("Lon size:"+str(lon.size))

    msg_info("...processing optics data from PreQC")
    y=np.fromfile(in_file, dtype=array_dtype,count=1)
    array=y['array']
    msg_debug("......statistics:")
    msg_debug(".........min:" + str(np.nanmin(array)))
    msg_debug(".........max:" + str(np.nanmax(array)))
    msg_debug("........mean:" + str(np.nanmean(array)))
    optics=np.array(array).byteswap().newbyteorder()
    msg_debug("Optics size:"+str(optics.size))

    #################################################################################
    #casting data-types from float 32 to int64 to support .query operation
    #################################################################################
    df=pd.DataFrame({'lat':lat[0,:], 'lon':lon[0,:], 'optics':optics[0,:]})
    lat=df['lat']
    df['lat'] = lat.astype('float', copy=True)
    lon=df['lon']
    df['lon'] = lon.astype('float', copy=True)
    optics=df['optics']
    df['optics'] = optics.astype('float', copy=True)

    df['Latitude']=df['lat']
    df['Longitude']=df['lon']

    #################################################################################
    #Pandas, filtering data down to focal area
    #################################################################################
    GOMEX=df.query(FLTR_DATA)

    #################################################################################
    #Debug output showing data reduction due to the filter
    #################################################################################
    msg_debug("Original dataset shape:"+ str(df.shape))
    msg_debug("   GOMEX dataset shape:"+ str(GOMEX.shape))
    msg_debug("   GOMEX lat dimension:" + str(GOMEX['lat'].size))
    msg_debug("   GOMEX lon dimension:" + str(GOMEX['lon'].size))

    #################################################################################
    #Create a map showing the total swath of the output
    #################################################################################
    show_map_coverage(df)
    #################################################################################
    #Create a map showing the filtered data set and focal point of analysis
    #################################################################################
    show_map_coverage(GOMEX)

    #################################################################################
    #Clean up unused variables
    #################################################################################
    del x
    del array
    del lat
    del lon
    del optics
    del df

    #################################################################################
    # NetCDF build after marshaling data.
    #################################################################################
    inc_lat_dim = GOMEX['lat'].size
    inc_lon_dim = GOMEX['lon'].size

    #close data file read in
    in_file.close()

    #################################################################################
    #NetCDF global attributes
    #################################################################################
    dataset = netcdf.netcdf_file(netcdf_filename, "w")
    meta_data_input="Visible Band Satellite Data to Improve Ocean Model Radiative Transfer (VISOR)"
    dataset.title = meta_data_input.encode(ENCODING, errors='ignore').strip()
    meta_data_input="Test validation for " + str(file)
    dataset.subtitle = meta_data_input.encode(ENCODING, errors='ignore')
    meta_data_input="Created with Jupyter Lab, NetCDF4 libraries, GOPS VISOR Parser input file."
    dataset.description = meta_data_input.encode(ENCODING, errors='ignore')
    meta_data_input="Created " + today.strftime("%d/%m/%y")
    dataset.history = meta_data_input.encode(ENCODING, errors='ignore')


    #################################################################################
    #NetCDF dimension declaration
    #################################################################################
    lat_dim = dataset.createDimension("lat", inc_lat_dim)
    lon_dim = dataset.createDimension("lon", inc_lon_dim)
    optics_dim = dataset.createDimension("optics", inc_lon_dim)
    x_dim = dataset.createDimension("x", inc_lat_dim)
    y_dim = dataset.createDimension("y", inc_lon_dim)


    #NetCDF Variable creation and data assignment
    #################################################################################
    # Latitude
    #################################################################################
    lat_reference = dataset.createVariable("lat", "f8", ("lat",))
    lat_reference.units="degrees north"
    lat_reference[:] = GOMEX['lat']

    #################################################################################
    # Longitude
    #################################################################################
    lon_reference = dataset.createVariable("lon", "f8", ("lon",))
    lon_reference.units="degrees east"
    lon_reference[:] = GOMEX['lon']

    #################################################################################
    # Optics Data
    #################################################################################
    optics_reference = dataset.createVariable("optics", "f8", ("optics",))
    optics_reference[:] = GOMEX['optics']
    optics_reference.units="m^-1"
    optics_reference.warning = str(product_name) + " generated from GOPS Parser, originated from GOPS processing system."

    dataset.close()
    msg_info(" ")
    msg_info("#####################################################################################################################")

[2024-08-27 23:12:26 UTC]   DEBUG: Processing 202001270400a for folderOnColab/merged.2020032.0201.L5.6D.a_490_lmi.nc_202001270400_a_490_lmi_2.out 


DEBUG:msg_logs:Processing 202001270400a for folderOnColab/merged.2020032.0201.L5.6D.a_490_lmi.nc_202001270400_a_490_lmi_2.out


[2024-08-27 23:12:26 UTC]   DEBUG: [(4, 26154810, 4, 4, 219, 4, 12, b'202001270400', 12)] 


DEBUG:msg_logs:[(4, 26154810, 4, 4, 219, 4, 12, b'202001270400', 12)]


[2024-08-27 23:12:26 UTC]   DEBUG: Total records to read in for each array:26154810 


...

DEBUG:msg_logs:   GOMEX dataset shape:(1433620, 5)


[2024-08-27 23:12:33 UTC]   DEBUG:    GOMEX lat dimension:1433620 


DEBUG:msg_logs:   GOMEX lat dimension:1433620


[2024-08-27 23:12:33 UTC]   DEBUG:    GOMEX lon dimension:1433620 


DEBUG:msg_logs:   GOMEX lon dimension:1433620
